<a href="https://colab.research.google.com/github/sedcakmak/Airline-Passenger-Satisfaction-Data-Analysis/blob/main/Airline_Passenger_Satisfaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install kagglehub


In [ ]:
import kagglehub

path = kagglehub.dataset_download("teejmahal20/airline-passenger-satisfaction")

print("Path to dataset files:", path)


Path to dataset files: /kaggle/input/airline-passenger-satisfaction


In [ ]:
import os

os.listdir('/kaggle/input/airline-passenger-satisfaction')


['train.csv', 'test.csv']

In [ ]:
import pandas as pd

train = pd.read_csv('/kaggle/input/airline-passenger-satisfaction/train.csv')
test = pd.read_csv('/kaggle/input/airline-passenger-satisfaction/test.csv')

df = pd.concat([train, test], ignore_index=True)
df.head(10)



,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied
5,5,111157,Female,Loyal Customer,26,Personal Travel,Eco,1180,3,4,...,1,3,4,4,4,4,1,0,0.0,neutral or dissatisfied
6,6,82113,Male,Loyal Customer,47,Personal Travel,Eco,1276,2,4,...,2,3,3,4,3,5,2,9,23.0,neutral or dissatisfied
7,7,96462,Female,Loyal Customer,52,Business travel,Business,2035,4,3,...,5,5,5,5,4,5,4,4,0.0,satisfied
8,8,79485,Female,Loyal Customer,41,Business travel,Business,853,1,2,...,1,1,2,1,4,1,2,0,0.0,neutral or dissatisfied
9,9,65725,Male,disloyal Customer,20,Business travel,Eco,1061,3,3,...,2,2,3,4,4,3,2,0,0.0,neutral or dissatisfied


In [ ]:
df.shape         # how many rows & columns
df.columns       # list of column names
df.info()        # data types + missing values
df.describe()    # basic stats (only numeric columns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129880 entries, 0 to 129879
Data columns (total 25 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Unnamed: 0                         129880 non-null  int64  
 1   id                                 129880 non-null  int64  
 2   Gender                             129880 non-null  object 
 3   Customer Type                      129880 non-null  object 
 4   Age                                129880 non-null  int64  
 5   Type of Travel                     129880 non-null  object 
 6   Class                              129880 non-null  object 
 7   Flight Distance                    129880 non-null  int64  
 8   Inflight wifi service              129880 non-null  int64  
 9   Departure/Arrival time convenient  129880 non-null  int64  
 10  Ease of Online booking             129880 non-null  int64  
 11  Gate location                      1298

,Unnamed: 0,id,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
count,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129487.000000
mean,44158.700000,64940.500000,39.427957,1190.316392,2.728696,3.057599,2.756876,2.976925,3.204774,3.252633,3.441361,3.358077,3.383023,3.350878,3.632114,3.306267,3.642193,3.286326,14.713713,15.091129
std,31207.377062,37493.270818,15.119360,997.452477,1.329340,1.526741,1.401740,1.278520,1.329933,1.350719,1.319289,1.334049,1.287099,1.316252,1.180025,1.266185,1.176669,1.313682,38.071126,38.465650
min,0.000000,1.000000,7.000000,31.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,16234.750000,32470.750000,27.000000,414.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,3.000000,3.000000,2.000000,0.000000,0.000000
50%,38963.500000,64940.500000,40.000000,844.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,3.000000,4.000000,3.000000,0.000000,0.000000
75%,71433.250000,97410.250000,51.000000,1744.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000,4.000000,5.000000,4.000000,5.000000,4.000000,12.000000,13.000000
max,103903.000000,129880.000000,85.000000,4983.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,1592.000000,1584.000000


In [ ]:
df.isnull().sum() #number of missing values


,0
Unnamed: 0,0
id,0
Gender,0
Customer Type,0
Age,0
Type of Travel,0
Class,0
Flight Distance,0
Inflight wifi service,0
Departure/Arrival time convenient,0


In [ ]:
# Eksik veri sayısı
print(df['Arrival Delay in Minutes'].isnull().sum())

# Eksiklerin toplam veri içindeki oranı
print(f"Oran: {df['Arrival Delay in Minutes'].isnull().mean():.2%}")

# Eksiklerin hangi satırlarda olduğunu görmek için örnek
print(df[df['Arrival Delay in Minutes'].isnull()].head())


393
Oran: 0.30%
      Unnamed: 0      id  Gender      Customer Type  Age   Type of Travel  \
213          213   49608  Female     Loyal Customer   38  Business travel   
1124        1124   73442    Male     Loyal Customer   53  Personal Travel   
1529        1529   71178    Male     Loyal Customer   39  Business travel   
2004        2004   72940  Female  disloyal Customer   26  Business travel   
2108        2108  116374  Female     Loyal Customer   24  Personal Travel   

         Class  Flight Distance  Inflight wifi service  \
213        Eco              109                      5   
1124       Eco             1012                      3   
1529  Business              733                      2   
2004  Business             1035                      3   
2108       Eco              417                      2   

      Departure/Arrival time convenient  ...  Inflight entertainment  \
213                                   3  ...                       5   
1124                        

In [ ]:
df.drop(['Unnamed: 0', 'id'], axis=1, inplace=True)


In [ ]:
df.dropna(subset=['Arrival Delay in Minutes'], inplace=True)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 129487 entries, 0 to 129879
Data columns (total 23 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Gender                             129487 non-null  object 
 1   Customer Type                      129487 non-null  object 
 2   Age                                129487 non-null  int64  
 3   Type of Travel                     129487 non-null  object 
 4   Class                              129487 non-null  object 
 5   Flight Distance                    129487 non-null  int64  
 6   Inflight wifi service              129487 non-null  int64  
 7   Departure/Arrival time convenient  129487 non-null  int64  
 8   Ease of Online booking             129487 non-null  int64  
 9   Gate location                      129487 non-null  int64  
 10  Food and drink                     129487 non-null  int64  
 11  Online boarding                    129487 no

In [ ]:
for col in df.select_dtypes(include=['object']).columns:
    print(f"--- {col} ---")
    print(df[col].unique()[:10])  # İlk 10 farklı değeri göster
    print()


--- Gender ---
['Male' 'Female']

--- Customer Type ---
['Loyal Customer' 'disloyal Customer']

--- Type of Travel ---
['Personal Travel' 'Business travel']

--- Class ---
['Eco Plus' 'Business' 'Eco']

--- satisfaction ---
['neutral or dissatisfied' 'satisfied']



In [ ]:
# Number of males and females
gender_counts = df['Gender'].value_counts()
for gender, count in gender_counts.items():
    print(f"{gender}: {count}")


# Average age
print("Average age:", round(df['Age'].mean(), 2))

# Most common passenger age
print("Most common passenger age:", df['Age'].mode()[0])

# Youngest passenger
print("Youngest age:", df['Age'].min())

# Oldest passenger
print("Oldest age:", df['Age'].max())

# Number of passengers under 18
print("Number of passengers under 18:", df[df['Age'] < 18].shape[0])


Female: 65703
Male: 63784
Average age: 39.43
Most common passenger age: 39
Youngest age: 7
Oldest age: 85
Number of passengers under 18: 9811


In [ ]:
# Create age bins
age_bins = pd.cut(df['Age'], bins=[0, 19, 29, 39, 49, 59, 69, 79, 89])
print(age_bins.value_counts().sort_values(ascending=False))



Age
(39, 49]    29555
(19, 29]    26098
(29, 39]    25623
(49, 59]    23902
(0, 19]     12141
(59, 69]    10442
(69, 79]     1591
(79, 89]      135
Name: count, dtype: int64
